In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import json
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
import torch
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator
import spacy
import en_core_web_sm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from tqdm import tqdm, trange
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
import time
from collections import Counter

path = "/content/drive/My Drive/Colab Notebooks/data/"


In [3]:
!pip install -U torch

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.4.0)


In [4]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [5]:
manual_seed = 77
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
    torch.cuda.manual_seed(manual_seed)

cuda


#EDA

In [0]:
# reading in 50k data
df = pd.read_csv(path+"subset_100k.csv", index_col=0, encoding="utf-8").reset_index(drop=True)
# df_shuffled = df.sample(frac=1, random_state=123) #shuffle rows randomly
# df_shuffled = df_shuffled.drop(columns="index") #drops index to only keep text and label
train, validate, test = np.split(df.sample(frac=1, random_state=123).drop(columns="index"), 
                                                                          [int(.6*len(df)), int(.8*len(df))])
train_texts, train_labels = zip(*train.values) #resulting type is tuples
valid_texts, valid_labels = zip(*validate.values)
test_texts, test_labels = zip(*test.values)

In [7]:
#check distribution of shuffled data by class (random state 123)

print(Counter(train_labels))
print(Counter(valid_labels))
print(Counter(test_labels))

Counter({0: 30118, 1: 29882})
Counter({1: 10124, 0: 9876})
Counter({0: 10006, 1: 9994})


In [8]:
print(train[:3])
print(train[-3:])

                                                    text  labels
42083  MUMBAI: India's star professional boxer and Ol...       0
71825  P.S: This may not seem like something that is ...       1
99535  This pattern is available\n\nThe pattern is fr...       1
                                                    text  labels
20737  Ohio Gov. John Kasich is continuing his attack...       0
37688  This page has been flagged for a review of its...       0
83502  An Ottawa family is speaking out after a photo...       1


In [9]:
print(validate[:3])
print(validate[-3:])

                                                    text  labels
75082  A recent study has found that children are mor...       1
88345  Greetings Friends,\n\n\nI hope everyone is cel...       1
95838  The UESPWiki – Your source for The Elder Scrol...       1
                                                    text  labels
89728  The Washington Post reports this week that the...       1
32245  Climate Change Protesters Canceled March in Co...       0
4458   When Canadiens general manager Marc Bergevin s...       0


In [10]:
print(test[:3])
print(test[-3:])

                                                    text  labels
21563  According to local services' estimates, about ...       0
67879  "I feel extremely lucky that I was at the righ...       1
86351  The Department of Homeland Security (DHS) is t...       1
                                                    text  labels
17730  Just checked into the speaker forum to look fo...       0
28030  SHOWCASE ! That Bastard Is Trying To Steal Our...       0
15725  Touch Of Malice Check List\n\nMausoleum\n\n1\n...       0


In [0]:
#create train, val, test subsets from 50k subset

# train.to_csv(path+"subset_50k_train.csv", index=False)
# validate.to_csv(path+"subset_50k_valid.csv", index=False)
# test.to_csv(path+"subset_50k_test.csv", index=False)

In [12]:
print(len(train))
print(len(validate))
print(len(test))

print(len(train_texts))
print(len(valid_texts))
print(len(test_texts))

60000
20000
20000
60000
20000
20000


#Logistic Regression Baseline

In [0]:
#50k subset

n_jobs=None
verbose=False

start = time.time()

vect = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_features=2**21)
train_features = vect.fit_transform(train_texts)
valid_features = vect.transform(valid_texts)
test_features = vect.transform(test_texts)

model = LogisticRegression(solver='liblinear')
params = {'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10, 100]} #changed from original code
search = GridSearchCV(model, params, cv=5, n_jobs=n_jobs, verbose=verbose)
search.fit(sparse.vstack([train_features, valid_features]), train_labels+valid_labels)
print(search.best_params_)
model = model.set_params(**search.best_params_)
model.fit(train_features, train_labels)
valid_accuracy = model.score(valid_features, valid_labels)*100.
test_accuracy = model.score(test_features, test_labels)*100.
data = {
    'valid_accuracy':valid_accuracy,
    'test_accuracy':test_accuracy
}

end = time.time()

print("time in seconds:", end-start)
print(data)

{'C': 100}
{'valid_accuracy': 85.79, 'test_accuracy': 87.01}


# Load and prepare dataset for pytorch

In [0]:
# spacy_en = spacy.load("en_core_web_sm", disable=["tagger", "parser", "ner","textcat"])
# spacy_en.add_pipe(spacy_en.create_pipe('sentencizer')) #adding sentence tokenizer

spacy_en = spacy.load('en_core_web_sm') #changed syntax?

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

TEXT = Field(sequential=True, tokenize=tokenize_en, lower=True)
LABEL = Field(sequential=False, unk_token = None)


In [0]:
#100k subset
train, val, test = TabularDataset.splits(
               path=path, # the root directory where the data lies
               train='subset_100k_train.csv', validation="subset_100k_valid.csv", test="subset_100k_test.csv", # file names
               format='csv',
               skip_header=True, 
               fields=[('text', TEXT), ('label', LABEL)])

In [17]:
#100k subset

TEXT.build_vocab(train, min_freq=2)
LABEL.build_vocab(train)

print("Vocabulary size of TEXT:",len(TEXT.vocab.stoi))
print("Vocabulary size of LABEL:",len(LABEL.vocab.stoi))

Vocabulary size of TEXT: 145549
Vocabulary size of LABEL: 2


In [0]:
train_iter, val_iter, test_iter = BucketIterator.splits( #don't we usually not do test_iter?????
 (train, val, test), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(64,256,256),
 sort_key=lambda x: len(x.text), 
 sort=True,
# A key to use for sorting examples in order to batch together examples with similar lengths and minimize padding. 
 sort_within_batch=True
)

In [18]:
# create a single batch and terminate the loop
i = 0
for batch in val_iter:
    texts = batch.text
    labels = batch.label
    print(i)
    print(texts)
    print(labels)
    i+=1
    if i ==3:
      break

    #break  #we use first batch as an example.
    
print('texts:', texts.shape)
print('labels:', labels.shape)

# texts: [length, batch size]
# labels: [batch size]

0
tensor([[    2,    11,    14,  ...,     0,    21, 11695],
        [  156,    24,    13,  ...,     8,   115,  2534],
        [    6,    13,   266,  ..., 17087,   101,  1805],
        ...,
        [  152,  9754,  2035,  ...,     1,     1,     1],
        [ 1035,     4,  1538,  ...,     1,     1,     1],
        [    4,    11,     4,  ...,     1,     1,     1]])
tensor([0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,

# CNN baseline

In [0]:
# To define a CNN class
class CNN_Text(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, output_size, kernel_num, region_sizes, dropout):
        '''
        vocabulary_size: vocabulary size
        embedding_dim: word embedding size
        output_size: number of classes in prediction
        kernel_num: number of kernels (number of output channels of convolutional layers)
        region_sizes: height of kernels of convolutional layers
        dropout: dropout rate
        '''
        super(CNN_Text, self).__init__()
        # the size of input channel is 1.
        Ci = 1
        
        # word embedding layer
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_dim )
        
        # convolution with kernels
        self.convolution_layers = nn.ModuleList([nn.Conv2d(in_channels = Ci, out_channels = kernel_num, kernel_size = (K, embedding_dim)) for K in region_sizes])
        
        # a dropout layer
        self.dropout = nn.Dropout(dropout) 
        
        # fully connected layer
        self.fc = nn.Linear(len(kernel_sizes) * kernel_num, output_size)

    def forward(self, x):
        # input x  [sequence length, batch size]
        
        input_embeddings = self.embeddings(x)  
        # (batch size, word_sequence, embedding_dim) word embedding

        input_embeddings = input_embeddings.permute(1,0,2)
        input_embeddings = input_embeddings.unsqueeze(1)
        #  [batch size, number of channel is one, sequence length, embeeding size]

        # convolutional layers
        convolute_outputs = [F.relu(conv(input_embeddings)).squeeze(3) for conv in self.convolution_layers]  
        
        # to get the maximum value of filtered tensor
        max_pooling_outputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in convolute_outputs] 
        
        concat_list = torch.cat(max_pooling_outputs, 1) # concatenate representations
        
        drop_output = self.dropout(concat_list)  # add drop layer
        
        fc1_output = self.fc(drop_output)  # get the fc1 using a fully connected layer
        
        final_output = F.softmax(fc1_output,dim=1)
        
        return final_output


In [0]:
# Hyper Parameters

# the vocabulary size
vocabulary_size = len(TEXT.vocab.stoi) 

# Dimension of word embedding is 300. Namely, each word is expressed by a vector that has 300 dimensions.
embedding_dim = 300 

# region size as 2, 3, and 4
kernel_sizes = [2,3,4] 

# the number of kernel in each region size
kernels_num = 32  

# The dropout rate is set to be 0.5.
dropout = 0.5

# The output size of labels.
output_size = 2

# learning rate is set to be 0.01. CHANGED FROM ORIGINAL
lr = 0.0001        

# The number of iteration is set to be 5.
num_epoch = 5  

# employ class CNN_Text and assign to cnn
model = CNN_Text(vocabulary_size, embedding_dim, output_size, kernels_num, kernel_sizes, dropout).to(device)

In [21]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.1)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

CNN_Text(
  (embeddings): Embedding(146142, 300)
  (convolution_layers): ModuleList(
    (0): Conv2d(1, 32, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 32, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 32, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=96, out_features=2, bias=True)
)

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 43,929,290 trainable parameters


In [0]:
# Loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # define a optimizer for backpropagation
loss_func = nn.CrossEntropyLoss()   # define loss funtion

In [0]:
def train(model, iterator, optimizer, criterion):
    
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        batch_input, labels = batch.text, batch.label
        batch_input = batch_input.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        
        outputs = model(batch_input)

        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.cpu().item()

    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    all_pred=[]
    all_label = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            batch_input, labels = batch.text, batch.label
            batch_input = batch_input.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(batch_input)

            loss = criterion(outputs, labels)

            epoch_loss += loss.cpu().item()

            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(labels.cpu())
    
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='macro') 
    return epoch_loss / len(iterator), accuracy, f1score

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
len(val_iter)

40

In [26]:
# 50k subset
MAX_EPOCH = 20
total_step = len(train_iter)
loss_list = []
acc_list = []

for epoch in trange(MAX_EPOCH, desc="Epoch"):
    start_time = time.time()
    train_loss = train(model, train_iter, optimizer, loss_func)  
    val_loss, val_acc, val_f1 = evaluate(model, val_iter, loss_func)
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    #torch.save(state, "./drive/My Drive/Colab Notebooks/ckpt_cnn/CNN_TEXT_"+str(epoch+1)+".pt")
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print('\n Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, 
                                                                MAX_EPOCH, train_loss, val_loss, val_acc, val_f1))
    

Epoch:   5%|▌         | 1/20 [00:35<11:19, 35.75s/it]

Epoch: 01 | Time: 0m 35s

 Epoch [1/20], Train Loss: 0.5524, Validation Loss: 0.5830, Validation Accuracy: 0.6923, Validation F1: 0.6713


Epoch:  10%|█         | 2/20 [01:10<10:37, 35.42s/it]

Epoch: 02 | Time: 0m 34s

 Epoch [2/20], Train Loss: 0.5374, Validation Loss: 0.5694, Validation Accuracy: 0.7110, Validation F1: 0.6945


Epoch:  15%|█▌        | 3/20 [01:44<09:57, 35.12s/it]

...

Epoch: 18 | Time: 0m 34s

 Epoch [18/20], Train Loss: 0.3446, Validation Loss: 0.4850, Validation Accuracy: 0.8123, Validation F1: 0.8093


Epoch:  95%|█████████▌| 19/20 [11:02<00:34, 34.91s/it]

Epoch: 19 | Time: 0m 35s

 Epoch [19/20], Train Loss: 0.3409, Validation Loss: 0.4890, Validation Accuracy: 0.8077, Validation F1: 0.8041


Epoch: 100%|██████████| 20/20 [11:36<00:00, 34.83s/it]

Epoch: 20 | Time: 0m 34s

 Epoch [20/20], Train Loss: 0.3383, Validation Loss: 0.4885, Validation Accuracy: 0.8085, Validation F1: 0.8051
